In [1]:
"""Write separate decorators that:
1. allows a function to run only on odd seconds - 100pts
2. log - 100pts
3. authenticate - 300pts
4. timed (n times) - 100pts
5. Provides privilege access (has 4 parameters, based on privileges (high, mid, low, no), gives access to all 4, 3, 2 or 1 params) - 200pts
Write our htmlize code using inbuild singledispatch - 100pts"""

'Write separate decorators that:\n1. allows a function to run only on odd seconds - 100pts\n2. log - 100pts\n3. authenticate - 300pts\n4. timed (n times) - 100pts\n5. Provides privilege access (has 4 parameters, based on privileges (high, mid, low, no), gives access to all 4, 3, 2 or 1 params) - 200pts\nWrite our htmlize code using inbuild singledispatch - 100pts'

In [156]:
import time
from time import perf_counter

def run_oddsec(fn):

    def check_run():
        check_timenow = round(perf_counter(),0)
        print('time checknow' ,check_timenow)

        if check_timenow % 2 == 0:
            print("Func runs now - even sec")
            fn()
    
    return check_run

In [160]:
@run_oddsec
def add():
    print (1+2)

In [161]:
type(add())

time checknow 1292.0
Func runs now - even sec
3


NoneType

In [168]:
# Log of functions

def logger(fn):
    '''Decorator takes care of function stats'''

    from functools import wraps
    stats_dict = dict()

    @wraps(fn)
    def store_stats():
        stats_dict["func_started"] = time.perf_counter()
        fn()
        stats_dict["func_ended"] = time.perf_counter()
        stats_dict["run_time"] = stats_dict["func_ended"]  - stats_dict["func_started"]
        stats_dict["func_name"] = fn.__name__
        stats_dict["func_docstr"] = fn.__doc__

        return stats_dict

    return store_stats

In [169]:
@logger
def add():
    print (1+2)

In [170]:
add()

3


{'func_started': 1843.1067292,
 'func_ended': 1843.1067681,
 'run_time': 3.889999993589299e-05,
 'func_name': 'add',
 'func_docstr': None}

In [193]:
# authenticate - 300pts

def authenticate(fn):
    '''Decorator to check for the authentication before accessing any functions'''

    def check_creds(*, user_password, in_password:str):
        if user_password() != hash(in_password):
            print('Bhai isko hack karke kya hi achive kar lega life mai')

        else:
            print('suer Authenticated')
            print(f"Function {fn.__name__} is called")
            return fn()
    return check_creds

In [198]:
authenticate(print, user_password)

TypeError: authenticate() takes 1 positional argument but 2 were given

In [192]:
def set_password(password=None):
    def inner():
        nonlocal password
        if password == None:
            password = input()
        return hash(password)
    return inner

In [197]:
user_password = set_password()

In [190]:
set_password()

-361540255091132360

In [172]:
def random_adminstuff():
    '''Provides privilege access (has 4 parameters, based on privileges (high, mid, low, no), gives access to all 4, 3, 2 or 1 params) - 200pts'''

    privilege = {"User":1, "Manager":2, "Admin":3, "SuperAdmin":4}


-6481112204484497659

In [32]:
import pandas as pd
data = { "Emp_Names":['Nikhil','Naman', 'Ad', 'SupAd'],
        "ID": [1,2,3,4],
        "Priviledge": [1,2,3,4 ], 
        "Money": ["Bohotkam", "Bohotzada", "Mat hi poocho", "Paisa kya hota hai"],
        "Useless_Info": ['sdfsadg', 'srtwtrew', 'kyukuiuy', 'gjkgjh']}

In [33]:
df = pd.DataFrame(data, columns = ['Emp_Names', "Priviledge", "Money", "ID", "Useless_Info"])

In [37]:
df.loc[df['Emp_Names'] == "Nikhil"]

,Emp_Names,Priviledge,Money,ID,Useless_Info
0,Nikhil,1,Bohotkam,1,sdfsadg


In [98]:
from functools import wraps
def sod(fn):
    '''Maps the user's and returns information applicable to him'''
    dict_priv = {
    1: ["Emp_Names"],
    2: ["Emp_Names", "Priviledge", "Money"],
    3: ["Emp_Names", "Priviledge", "Money", "ID"],
    4: ["Emp_Names", "Priviledge", "ID", "Useless_Info"]}
    
    @wraps(fn)
    def check_priv(emp_name):
        '''Returns dataframe output based on user's priviledges'''
        if emp_name in df.values:
            priv_no = df.loc[df['Emp_Names'] == emp_name]["Priviledge"]
            priv_no = (priv_no.item())
            return df[dict_priv[priv_no]]
        else:
            print('Employee data not in system, please double check')
    return check_priv

In [99]:
@sod
def check_userdata(emp_name):
    print("abc")

In [100]:
check_userdata("Nikil")

Employee data not in system, please double check


In [91]:
# Timed function ( n times )

def timed(reps):
    

False

In [30]:
@htmlize.register(float)
def html_real(a):
    return f'{round(a, 2)}'

from decimal import Decimal
@htmlize.register(Decimal)
def html_real(a):
    return f'{round(a, 2)}'

def html_escape(arg):
    return escape(str(arg))

@htmlize.register(str)
def html_str(s):
    return html_escape(s).replace('\n', '<br/>\n')

@htmlize.register(tuple)
@htmlize.register(list)
def html_sequence(l):
    items = (f'<li>{html_escape(item)}</li>' for item in l)
    return '<ul>\n' + '\n'.join(items) + '\n</ul>'

@htmlize.register(dict)
def html_dict(d):
    items = (f'<li>{k}={v}</li>' for k, v in d.items())
    return '<ul>\n' + '\n'.join(items) + '\n</ul>'

NameError: name 'Emp_Names' is not defined